Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [146]:
df['SALE_PRICE'][600]

380000

In [0]:
# df['LAND_SQUARE_FEET'] = (
#     df['LAND_SQUARE_FEET']
#     .str.replace(',','')
#     .astype(float)
# )

In [148]:
df.dtypes

BOROUGH                             int64
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [149]:
df['BOROUGH'].value_counts()

4    7494
3    6647
1    4803
2    2188
5    1908
Name: BOROUGH, dtype: int64

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [151]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

top10

Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'UPPER WEST SIDE (79-96)'],
      dtype='object')

In [0]:


# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [153]:
df['NEIGHBORHOOD'].value_counts()

OTHER                      19117
FLUSHING-NORTH               685
UPPER EAST SIDE (59-79)      465
UPPER EAST SIDE (79-96)      410
BEDFORD STUYVESANT           408
BOROUGH PARK                 376
UPPER WEST SIDE (59-79)      350
GRAMERCY                     323
ASTORIA                      312
FOREST HILLS                 298
UPPER WEST SIDE (79-96)      296
Name: NEIGHBORHOOD, dtype: int64

### Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.

In [154]:
# How many obeservations have '01 ONE FAMILY DWELINGS'

(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS').value_counts()

False    17979
True      5061
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [155]:
# How Many observations have a sale price between $100,000 and $2,000,000

((df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)).value_counts()

True     13411
False     9629
Name: SALE_PRICE, dtype: int64

In [156]:
# Get the subset of data where both the above conditions are TRUE

df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
        (df['SALE_PRICE'] > 100000) &
        (df['SALE_PRICE'] < 2000000)]

df.shape

(3151, 21)

In [0]:
# Change the LAND_SQUARE_FEET from string to float

df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .str.replace(',','')
    .astype(int)
)


In [158]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                    int64
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')

In [160]:
# create the train and test split data

train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]

print(train.shape)
print(test.shape)
print(df.shape)

(2507, 21)
(644, 21)
(3151, 21)


### Do one-hot encoding of categorical features.

In [161]:
df.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                           int64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [162]:
# Look at the unique counts of other non numeric columns to see if there are high cardinality
df.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,3151,3151,3151,3151,3151,3151,1,3151,3151
unique,5,6,1,2,13,3135,1,11,91
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,267 DECKER AVENUE,RP.,A1,2019-01-31 00:00:00
freq,1580,2990,3151,3111,1185,2,1,1186,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-30 00:00:00


In [164]:
#what about null values? it looks like Easement is nUll for all values...lets drop that column to0

df['EASE-MENT'].isnull().value_counts()

True    3151
Name: EASE-MENT, dtype: int64

In [0]:
# Looks like there are a couple 'ADDRESS', and 'SALE_DATE' that have some high cardinality.
# we will drop those columns from the dataset

target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'SALE_DATE', 'EASE-MENT']
features = train.columns.drop([target] + high_cardinality)


In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [167]:
df.shape, X_train.shape, X_test.shape

((3151, 21), (2507, 17), (644, 17))

In [0]:
# import the encoder for one hot key encode

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [169]:
X_train.shape

(2507, 50)

In [0]:
#now that we have fit_transformed X_train lets transorm X_test

X_test = encoder.transform(X_test)

In [171]:
X_test.shape

(644, 50)

In [0]:
X_test.isnull().sum()

In [0]:
X_train.isnull().sum()

### Do feature selection with `SelectKBest`.

In [0]:
# Import from sklearn

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [176]:
selector = SelectKBest(score_func=f_regression, k=26)

X_train_selected = selector.fit_transform(X_train, y_train)
X_train_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


(2507, 26)

In [177]:
# Which features did it select?

selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
TAX_CLASS_AT_PRESENT_1
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_S0
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_S0


Features not selected:
BOROUGH_1
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_B3
APARTMENT_N

In [0]:
# Its interesting that we are including certain buroghs and exlcuding others
# also the same thing applies to building classes... That is what happens with 
# one hot encoding tho when we are creating a ton more features. 

In [179]:
#set the selctor and see the Mean absolute error for each n number of features. 

for k in range(1, len(X_train.columns)+1):
    print(k, 'features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: $', mae,  '\n')

1 features
Test Mean Absolute Error: $ 183640.5858012459 

2 features
Test Mean Absolute Error: $ 179554.76843033519 

3 features
Test Mean Absolute Error: $ 179291.46658251373 

4 features
Test Mean Absolute Error: $ 179291.4665825138 

5 features
Test Mean Absolute Error: $ 170482.97112181393 

6 features
Test Mean Absolute Error: $ 169982.36848242735 

7 features
Test Mean Absolute Error: $ 168139.67298279447 

8 features
Test Mean Absolute Error: $ 168244.54060436622 

9 features
Test Mean Absolute Error: $ 167855.351270976 

10 features
Test Mean Absolute Error: $ 164737.0381194645 

11 features
Test Mean Absolute Error: $ 165346.03301324436 

12 features
Test Mean Absolute Error: $ 164860.22749045547 

13 features
Test Mean Absolute Error: $ 155159.4046262355 

14 features
Test Mean Absolute Error: $ 156541.34796777202 

15 features
Test Mean Absolute Error: $ 156571.5945132 

16 features
Test Mean Absolute Error: $ 156572.86128856236 

17 features
Test Mean Absolute Error: $ 156

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Test Mean Absolute Error: $ 156255.3168658793 

21 features
Test Mean Absolute Error: $ 154395.609991336 

22 features
Test Mean Absolute Error: $ 154425.7139719727 

23 features
Test Mean Absolute Error: $ 154425.71397197372 

24 features
Test Mean Absolute Error: $ 154425.71397198163 

25 features
Test Mean Absolute Error: $ 154425.71397198542 

26 features
Test Mean Absolute Error: $ 154121.52407650973 

27 features
Test Mean Absolute Error: $ 153987.11458785058 

28 features
Test Mean Absolute Error: $ 153911.35651236202 

29 features
Test Mean Absolute Error: $ 153862.7356579448 

30 features
Test Mean Absolute Error: $ 154713.06515626097 

31 features
Test Mean Absolute Error: $ 154838.93149346666 

32 features
Test Mean Absolute Error: $ 154838.9314934713 

33 features
Test Mean Absolute Error: $ 154787.79552005953 

34 features
Test Mean Absolute Error: $ 154781.2495773194 

35 features
Test Mean Absolute Error: $ 154760.23157893025 

36 features
Test Mean Absolute Error: $ 154

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

In [180]:
#what about the coefficients?

pd.DataFrame(model.coef_.T, index=X_train.columns, columns=['Coefficients'])

,Coefficients
BOROUGH_3,5.542629e+13
BOROUGH_4,5.542629e+13
BOROUGH_2,5.542629e+13
BOROUGH_5,5.542629e+13
BOROUGH_1,5.542629e+13
NEIGHBORHOOD_OTHER,-8.504920e+04
NEIGHBORHOOD_FLUSHING-NORTH,3.026017e+04
NEIGHBORHOOD_BEDFORD STUYVESANT,-4.185385e+05
NEIGHBORHOOD_FOREST HILLS,2.760196e+05
NEIGHBORHOOD_BOROUGH PARK,1.467769e+05


### Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)

In [0]:
# Import Ridge regression model from sklearn
from sklearn.linear_model import Ridge

In [0]:
# set the model 

model = Ridge()